In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import ResNet50, DenseNet121, EfficientNetB3
from keras.optimizers import Adam
from keras.losses import CategoricalCrossentropy
import os, cv2, json
import glob

import warnings
warnings.simplefilter("ignore")

In [ ]:
WORK_DIR = 'E:/Kaggle.Challenges/cassava-leaf-disease-classification/'
os.listdir(WORK_DIR)

In [ ]:
with open('E:/Kaggle.Challenges/cassava-leaf-disease-classification/label_num_to_disease_map.json', 'r') as file:
    labels = json.load(file)
    
labels

In [ ]:
data=pd.read_csv(WORK_DIR + '/train.csv')

In [ ]:
data.head()

In [ ]:
data.dtypes

In [ ]:
#change for the ImageDataGen and flow_from_dataframe
data.label=data.label.astype('str')

In [ ]:
data.dtypes

In [ ]:
data.shape[0]

In [ ]:
data.label.value_counts()

# Image Visualization

In [ ]:
IMG_SIZE = 333

In [ ]:
plt.figure(figsize=(15,12))
data_sample=data.sample(9).reset_index(drop=True)

for i in range(8):
  plt.subplot(2,4,i+1)

  img=cv2.imread(WORK_DIR + 'train_images/' + data_sample.image_id[i])
  img=cv2.resize(img, (IMG_SIZE, IMG_SIZE))
  img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  plt.axis("off")
  plt.imshow(img)
  plt.title(labels.get(data_sample.label[i]))

plt.tight_layout()
plt.show()

In [ ]:
data_sample

# Cassava Bacterial Blight (CBB)

In [ ]:
labels.get('0')

In [ ]:
plt.figure(figsize=(15,12))
data_sample=data[data.label=='0'].sample(4).reset_index(drop=True)

for i in range(4):
  plt.subplot(1,4,i+1)

  img=cv2.imread(WORK_DIR+'train_images/'+data_sample.image_id[i])
  img=cv2.resize(img, (IMG_SIZE, IMG_SIZE))
  img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

  plt.axis('off')
  plt.imshow(img)
  plt.title(labels.get(data_sample.label[i]))

plt.tight_layout()
plt.show()

**Cassava Brown Streak Disease (CBSD)**

In [ ]:
labels.get('1')

In [ ]:
plt.figure(figsize=(15,12))
data_sample = data[data.label=="1"].sample(4).reset_index(drop=True)
for i in range(4):
    plt.subplot(1,4,i+1)
    
    img = cv2.imread(WORK_DIR + "train_images/" + data_sample.image_id[i])
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plt.axis("off")
    plt.imshow(img)
    plt.title(labels.get(data_sample.label[i]))
    
plt.tight_layout()
plt.show()

**Cassava Green Mottle (CGM)**

In [ ]:
labels.get("2")

In [ ]:
plt.figure(figsize=(15,12))
data_sample = data[data.label=="2"].sample(4).reset_index(drop=True)
for i in range(4):
    plt.subplot(1,4,i+1)
    
    img = cv2.imread(WORK_DIR + "train_images/" + data_sample.image_id[i])
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plt.axis("off")
    plt.imshow(img)
    plt.title(labels.get(data_sample.label[i]))
    
plt.tight_layout()
plt.show()

**Cassava Mosaic Disease (CMD)**

In [ ]:
labels.get("3")

In [ ]:
plt.figure(figsize=(15,12))
data_sample = data[data.label=="3"].sample(4).reset_index(drop=True)
for i in range(4):
    plt.subplot(1,4,i+1)
    
    img = cv2.imread(WORK_DIR + "train_images/" + data_sample.image_id[i])
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plt.axis("off")
    plt.imshow(img)
    plt.title(labels.get(data_sample.label[i]))
    
plt.tight_layout()
plt.show()

**Healthy**

In [ ]:
labels.get("4")

In [ ]:
plt.figure(figsize=(15,12))
data_sample = data[data.label=="4"].sample(4).reset_index(drop=True)
for i in range(4):
    plt.subplot(1,4,i+1)
    
    img = cv2.imread(WORK_DIR + "train_images/" + data_sample.image_id[i])
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plt.axis("off")
    plt.imshow(img)
    plt.title(labels.get(data_sample.label[i]))
    
plt.tight_layout()
plt.show()

**Image Preprocessing, Data Augmentation**

**ImageDataGenerator**: Generate batches of tensor image data with real-time data augmentation

**flow_from_dataframe**: Takes the dataframe and the path to a directory + generates batches. The generated batches contain augmentaed/normalized data.

In [ ]:
train_generator=ImageDataGenerator(rotation_range=270,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   brightness_range=[0.1,0.9],
                                   shear_range=25,
                                   zoom_range=0.3,
                                   channels_shift_range=0.2,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   validation_split=0.2).flow_from_dataframe(data,
                                                                             directory=WORK_DIR+'train_images',
                                                                             x_col='image_id',
                                                                             y_col='label',
                                                                             target_size=(IMG_SIZE, IMG_SIZE),
                                                                             class_mode='categorical',
                                                                             batch_size=32,
                                                                             shuffle=True,
                                                                             subset='training')

In [ ]:
valid_generator=ImageDataGenerator(validation_split=0.2).flow_from_dataframe(data,
                                                                             directory=WORK_DIR+'train_images',
                                                                             x_col='image_id',
                                                                             y_col='label',
                                                                             target_size=(IMG_SIZE, IMG_SIZE),
                                                                             class_mode='categorical',
                                                                             batch_size=32,
                                                                             shuffle=True,
                                                                             subset='validation')

In [ ]:
valid_generator.class_indices

**Model**

In [ ]:
def modelEfficientNetB3():
  model=models.Sequential()
  model.add(EfficientNetB3(include_top=False, weights='imagenet',
                           (IMG_SIZE, IMG_SIZE, 3)))
  model.add(layers.GlobalAveragePooling2D())
  model.add(layers.Dense(256, activation='relu'))
  model.add(layers.Dropout(0,5))
  model.add(layers.Dense(5, activation='softmax'))
  return model



In [ ]:
model = EfficientNetB3()

In [ ]:
model.summary()

In [ ]:
from tensor.keras import utils
utils.plot_model(model)

**Callbacks**

**ModelCheckpoint**: Callback to save the keras model or model weights at some frequency.

**EarlyStopping**: Stop training when a monitored metric has stopped improving.

**ReduceLROnPlateau**: Reduce learning rate when a metric has stopped improving.

In [ ]:
model_check=ModelCheckpoint('./FirstTry.h5',
                            monitor='val_loss',
                            verbose=1,
                            save_best_only=True,
                            save_weights_only=False,
                            mode='min'))

In [ ]:
early_stop=EarlyStopping(monitor='val_loss',
                         min_delta=0.001,
                         patience=7,
                         verbose=1,
                         mode='min',
                         restore_best_weights=False)

In [ ]:
reduce_lr=ReduceLROnPlateau(monitor='val_loss',
                            factor=0.1,
                            patience=2,
                            verbose=1,
                            mode='min',
                            min_delta=0.0001)

In [ ]:
model.compile(optimizer='adam',
              loss=CategoricalCrossentropy(label_smoothing=0.3),
                                           reduction='auto',
              name='categorical_crossentropy',
              metric=['accuracy'])

In [ ]:
history=model.fit_generator(train_generator,
                            epochs=30,
                            validation_data=valid_generator,
                            callbacks=[model_check,early_stop,reduce_lr])

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history.history['accuracy'], '*b-', label='train_acc')
plt.plot(history.history['val_accuracy'], 'r*-', label='val_acc')
plt.grid()
plt.title('train_acc vs val_acc')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
plt.plot(history.history['loss'], '*b-', label='train_loss')
plt.plot(history.history['val_loss'], 'r*-', label='val_loss')
plt.grid()
plt.title('train_loss vs val_loss')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend()
plt.show()